In [1]:
import numpy as np

def calculate_sigma(p_center, p_value, distance):
    """
    Calculate sigma (standard deviation) for a 2D Gaussian based on p_center, p_value, and distance.
    
    Parameters:
    - p_center: float, maximum connection probability at center (0,0)
    - p_value: float, connection probability at distance r
    - distance: float, the distance r at which p_value is measured (unit consistent with your model)
    
    Returns:
    - sigma: float, the standard deviation
    """
    if p_value >= p_center:
        raise ValueError("p_value must be smaller than p_center.")
    if p_value <= 0 or p_center <= 0:
        raise ValueError("p_value and p_center must both be positive.")

    sigma = distance / np.sqrt(-2 * np.log(p_value / p_center))
    return sigma


In [3]:
calculate_sigma(0.1, 0.064, (0.816-0.2735))

np.float64(0.5742190238754875)

In [15]:
import numpy as np
from scipy.integrate import dblquad
from scipy.optimize import minimize_scalar

def average_prob_given_sigma(sigma, p_center, size_um=1000.0):
    """
    在 1x1mm 的区域中，计算给定 sigma 时的平均连接概率
    """
    def integrand(x, y):
        return np.exp(-(x**2 + y**2) / (2 * sigma**2))
    
    # 积分范围从 -500um 到 +500um
    L = size_um / 2
    integral, _ = dblquad(integrand, -L, L, lambda x: -L, lambda x: L)
    area = size_um**2
    avg_p = p_center * integral / area
    return avg_p

def find_sigma_from_avg(avg_p, p_center, size_um=1000.0):
    """
    给定平均连接概率 avg_p 和最大连接概率 p_center，反推出高斯分布 sigma
    """
    def objective(sigma):
        if sigma <= 0:
            return np.inf
        pred = average_prob_given_sigma(sigma, p_center, size_um)
        return abs(pred - avg_p)
    
    result = minimize_scalar(objective, bounds=(10, 1000), method='bounded')
    return result.x

# 示例：图中某个连接为 25% × 34% = 0.085，p_center = 0.1
p_center = 0.1
avg_p = 0.064 # 表中值

sigma = find_sigma_from_avg(avg_p, p_center)
print(f"反推出的 sigma ≈ {sigma:.2f} μm")


反推出的 sigma ≈ 411.40 μm


In [1]:
Psg_params_list = [
        [{'rate': 100.0}, {'rate': 0.0}, {'rate': 0.0}, {'rate': 0.0}],
        [{'rate': 0.0}, {'rate': 100.0}, {'rate': 0.0}, {'rate': 0.0}],
        [{'rate': 0.0}, {'rate': 0.0}, {'rate': 100.0}, {'rate': 0.0}],
        [{'rate': 0.0}, {'rate': 0.0}, {'rate': 0.0}, {'rate': 100.0}]
    ]

In [2]:
Psg_params_list[1]

[{'rate': 0.0}, {'rate': 100.0}, {'rate': 0.0}, {'rate': 0.0}]